# Data

In [1]:
from src.mtal.data_collect import get_spot_pairs

cryptos = get_spot_pairs()

In [7]:
from src.mtal.data_collect import get_pair_df

# df = get_pair_df(pair="BTCUSDT")
df = get_pair_df(pair="BTCUSDT", limit=59, frequency="1d")
df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time
0,2024-03-01,61130.99,63114.23,60777.00,62387.90,47737.93473,2024-03-01 23:59:59.999
1,2024-03-02,62387.90,62433.19,61561.12,61987.28,25534.73659,2024-03-02 23:59:59.999
2,2024-03-03,61987.28,63231.88,61320.00,63113.97,28994.90903,2024-03-03 23:59:59.999
3,2024-03-04,63113.97,68499.00,62300.00,68245.71,84835.16005,2024-03-04 23:59:59.999
4,2024-03-05,68245.71,69000.00,59005.00,63724.01,132696.78130,2024-03-05 23:59:59.999


# EMA backtest

In [3]:
from src.mtal.utils import generate_pinescript
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester 

short_ema = 9 
long_ema = 50 
tester = MACrossBacktester(
    short_ma=short_ema, long_ma=long_ema, data=df
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)
print(generate_pinescript(entry_dates, exit_dates))


BacktestResults(pnl=1496.9455027530491, pnl_percentage=0.49694550275304916, max_drawdown=-0.22674807108058337, win_rate=0.4, average_return=0.06150325385921884, trade_number=10, entry_dates=[Timestamp('2021-11-15 00:00:00'), Timestamp('2022-03-23 00:00:00'), Timestamp('2022-08-11 00:00:00'), Timestamp('2022-10-29 00:00:00'), Timestamp('2023-01-12 00:00:00'), Timestamp('2023-03-15 00:00:00'), Timestamp('2023-06-22 00:00:00'), Timestamp('2023-08-09 00:00:00'), Timestamp('2023-10-02 00:00:00'), Timestamp('2024-01-31 00:00:00')], exit_dates=[Timestamp('2021-11-16 00:00:00'), Timestamp('2022-04-13 00:00:00'), Timestamp('2022-08-20 00:00:00'), Timestamp('2022-11-10 00:00:00'), Timestamp('2023-03-10 00:00:00'), Timestamp('2023-05-13 00:00:00'), Timestamp('2023-08-05 00:00:00'), Timestamp('2023-08-17 00:00:00'), Timestamp('2024-01-23 00:00:00'), Timestamp('2024-04-27 00:00:00')], entry_prices=[65519.11, 42364.13, 23954.05, 20591.84, 17943.26, 24670.41, 29993.89, 29770.41, 27992.58, 42941.1], e

In [4]:
from itertools import combinations
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester


ema_values = range(1, 60)  

combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

results = {}

for short_ema, long_ema in combinations:
    tester = MACrossBacktester(short_ma=short_ema, long_ma=long_ema, data=df)
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)  # Ajuster le critère si nécessaire
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")


Meilleure combinaison: Short EMA = 55, Long EMA = 59
Résultat du test: BacktestResults(pnl=2697.621393230537, pnl_percentage=1.6976213932305368, max_drawdown=0.25442455932371505, win_rate=1.0, average_return=0.7024548572963697, trade_number=2, entry_dates=[Timestamp('2023-01-19 23:59:59.999000'), Timestamp('2023-10-22 23:59:59.999000')], exit_dates=[Timestamp('2023-08-23 23:59:59.999000'), Timestamp('2024-04-25 23:59:59.999000')], entry_prices=[21071.59, 29992.46], exit_prices=[26432.72, 64498.34])


# VWMA Backtest

In [8]:
from src.mtal.utils import generate_pinescript
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester 

short_ema = 2
long_ema = 4
tester = MACrossBacktester(short_ma=short_ema, long_ma=long_ema, data=df, ma_type="vwma"
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)
# print(generate_pinescript(entry_dates, exit_dates))



BacktestResults(pnl=652.9981784463162, pnl_percentage=0.0006529981784463161, max_drawdown=-0.08159385521885525, win_rate=0.6666666666666666, average_return=0.0010693331240334955, trade_number=6, entry_dates=[Timestamp('2024-03-08 00:00:00'), Timestamp('2024-03-22 00:00:00'), Timestamp('2024-03-25 00:00:00'), Timestamp('2024-04-01 00:00:00'), Timestamp('2024-04-05 00:00:00'), Timestamp('2024-04-20 00:00:00')], exit_dates=[Timestamp('2024-03-16 00:00:00'), Timestamp('2024-03-23 00:00:00'), Timestamp('2024-03-31 00:00:00'), Timestamp('2024-04-03 00:00:00'), Timestamp('2024-04-11 00:00:00'), Timestamp('2024-04-25 00:00:00')], entry_prices=[66823.18, 65501.28, 67210.0, 71280.0, 68487.8, 63818.01], exit_prices=[69499.84, 63796.64, 69582.17, 65463.99, 70631.08, 64289.58], profit_pct_history=[0.04005586085546967, -0.02602452959697886, 0.03529489659276891, -0.08159385521885525, 0.03129433271327154, 0.007389293398524957], profit_history=[40055.86085546967, -27066.964533344497, 35753.33834531251,

In [6]:
from itertools import combinations
from src.mtal.backtesting.ma_cross_backtest import MACrossBacktester


ema_values = range(1, 60)  

combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

results = {}

for short_ema, long_ema in combinations:
    tester = MACrossBacktester(short_ma=short_ema, long_ma=long_ema, data=df, ma_type="vwma")
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

best_combination = max(results, key=lambda x: results[x].pnl)  # Ajuster le critère si nécessaire
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")


/home/romain/mtal/src/mtal/analysis.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[get_ma_names(span, "vwma")] = vwma
/home/romain/mtal/src/mtal/analysis.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[get_ma_names(span, "vwma")] = vwma
/home/romain/mtal/src/mtal/analysis.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragment

Meilleure combinaison: Short EMA = 56, Long EMA = 58
Résultat du test: BacktestResults(pnl=3577.2975760474555, pnl_percentage=2.5772975760474557, max_drawdown=-0.17333902100011148, win_rate=0.6, average_return=0.1732078101534478, trade_number=10, entry_dates=[Timestamp('2022-03-19 23:59:59.999000'), Timestamp('2022-08-12 23:59:59.999000'), Timestamp('2022-11-02 23:59:59.999000'), Timestamp('2023-01-06 23:59:59.999000'), Timestamp('2023-01-09 23:59:59.999000'), Timestamp('2023-04-17 23:59:59.999000'), Timestamp('2023-04-20 23:59:59.999000'), Timestamp('2023-07-05 23:59:59.999000'), Timestamp('2023-10-14 23:59:59.999000'), Timestamp('2024-02-06 23:59:59.999000')], exit_dates=[Timestamp('2022-04-04 23:59:59.999000'), Timestamp('2022-09-13 23:59:59.999000'), Timestamp('2022-11-05 23:59:59.999000'), Timestamp('2023-01-07 23:59:59.999000'), Timestamp('2023-04-16 23:59:59.999000'), Timestamp('2023-04-18 23:59:59.999000'), Timestamp('2023-05-28 23:59:59.999000'), Timestamp('2023-08-18 23:59:59

27, 28 vwma daily cross marche bien, même en bear !
9, 10 en bear gagne meme bien en bear.
Avec la strat "on rentre que le close est supérieur au prix", ça nous donne 1,2 comme best values, autrement dit "si la ligne close crossover".